### To predict the movement of stock prices, LSTM model is used. Here, we are only predicting the movement of closing price but the similar approach can be followed for other columns as well.
### The model is trained on the stock prices of AAL company which is then tested on other companies.
### As this is a time series prediction, we are considering the prices of past 60 days to predict the future price. 

In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# loading the stock prices of all companies in a dataframe
dataset = pd.read_csv('/content/stocks_in_recent_5_years.csv')

In [ ]:
data = dataset.drop(columns="adj_close")
data.head()

,date,open,high,low,close,volume,name
0,2017-05-30,48.580002,47.790001,48.320000,47.959999,5512900.0,AAL
1,2017-05-31,48.480000,47.529999,48.200001,48.410000,4486100.0,AAL
2,2017-06-01,49.360001,48.299999,48.500000,49.049999,4421400.0,AAL
3,2017-06-02,50.470001,49.369999,49.560001,49.520000,7708600.0,AAL
4,2017-06-05,49.950001,49.400002,49.529999,49.740002,5466700.0,AAL


In [ ]:
dataset.info()

In [ ]:
# getting the list of all companies 
companies = dataset.Name.unique()
companies

In [ ]:
# since AAL company is used for training, we are creating a new dataframe with AAL parameters
stock = dataset.loc[dataset['Name'] == 'DIS']
stock.info()

In [ ]:
stock.head()

In [ ]:
# creating an array with closing prices
training_set = stock[['close']] 

In [ ]:
training_set.head()

In [ ]:
training_set['close'].plot()
plt.ylabel('Stock Price')
plt.xlabel(None)
plt.title('Closing Prices of DIS')
plt.legend()
plt.show()

In [ ]:
# normalizing the values
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
training_set_scaled.shape

### In the below cell, we are creating 2 arrays, x_train and y_train.
* x_train stores the values of closing prices of past 365(or as specified in timestamp) days
* y_train stores the values of closing prices of the present day

In [ ]:
x_train = []
y_train = []
timestamp = 365
length = len(training_set)
for i in range(timestamp, length):
    x_train.append(training_set_scaled[i-timestamp:i, 0])
    y_train.append(training_set_scaled[i, 0])
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
print (x_train[0])
print ('\n')
print (y_train[0])

In [ ]:
x_train.shape

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
test_set = dataset.loc[dataset['Name'] == 'AAL']   # change CBS to whatever company from the list
test_set = test_set.loc[:, test_set.columns == 'close']

In [ ]:
# storing the actual stock prices in y_test starting from 60th day as the previous 60 days are used to predict the present day value.
y_test = test_set.iloc[timestamp:, 0:].values

In [ ]:
# storing all values in a variable for generating an input array for our model 
closing_price = test_set.iloc[:, 0:].values
closing_price_scaled = sc.transform(closing_price)

In [ ]:
# the model will predict the values on x_test
x_test = [] 
length = len(test_set)

for i in range(timestamp, length):
    x_test.append(closing_price_scaled[i-timestamp:i, 0])
    
x_test = np.array(x_test)
x_test.shape

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

def build_model():
  model = Sequential()

  model.add(LSTM(units = 92, return_sequences = True, input_shape = (x_train.shape[1], 1)))
  

  model.add(LSTM(units = 92, return_sequences = True))
  

  model.add(LSTM(units = 92, return_sequences = True))
  

  model.add(LSTM(units = 92, return_sequences = False))
  

  model.add(Dense(units = 1))

  model.compile(optimizer = 'adagrad', loss = 'mean_squared_error')
  return model


In [ ]:
model = build_model()

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=25)

## Now the model is trained. We will test the performance of our model by plotting the predicted stock prices and actual stock prices of other companies

In [ ]:
# Predict the stock prices
y_predict = model.predict(x_test)
predicted_price = sc.inverse_transform(y_predict)

In [ ]:
# plotting the results
plt.plot(y_test, color = 'blue', label = 'Actual Stock Price')
plt.plot(predicted_price, color = 'red', label = 'Predicted Stock Price')
plt.title('DIS Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

## Optimizing Performance

### - Dropout

In [ ]:
def build_model():
  model = Sequential()

  model.add(LSTM(units = 92, return_sequences = True, input_shape = (x_train.shape[1], 1)))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = False))
  model.add(Dropout(0.2))

  model.add(Dense(units = 1))

  model.compile(optimizer = 'adagrad', loss = 'mean_squared_error')
  return model

In [ ]:
model = build_model()

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=25)



### - Early Stopping



In [ ]:
def build_model():
  model = Sequential()

  model.add(LSTM(units = 92, return_sequences = True, input_shape = (x_train.shape[1], 1)))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = False))
  model.add(Dropout(0.2))

  model.add(Dense(units = 1))

  model.compile(optimizer = 'adagrad', loss = 'mean_squared_error')
  return model

In [ ]:
model = build_model()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.00001,
    patience=20,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=25, callbacks=callback)

### - Activation Function

In [ ]:
def build_model():
  model = Sequential()

  model.add(LSTM(units = 92, return_sequences = True, input_shape = (x_train.shape[1], 1)))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(LSTM(units = 92, return_sequences = False))
  model.add(Dropout(0.2))

  model.add(Dense(units = 1))

  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  return model

In [ ]:
model = build_model()

NameError: ignored

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=25)

In [ ]:
model.save('StockPrediction.h5')

In [ ]:
# model evaluation
from keras.models import load_model

model = load_model('StockPrediction.h5')

####